In [1]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import os
from torch.utils.data import Dataset, DataLoader
from os.path import join
from torchvision.transforms import v2
from torch import optim, nn
from torchmetrics.image import PeakSignalNoiseRatio
import torch
import torch.nn.functional as F
from tqdm import tqdm

In [2]:
image_transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])

label_transform=  v2.Compose([v2.ToImage(),  v2.ToDtype(torch.float32, scale=True)])

In [3]:
class ImageEnhancementDataset(Dataset):
    def __init__(self, image_folder, label_folder, input_transform=None, label_transform=None):
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.input_transform = input_transform
        self.label_transform = label_transform

        # Ensure both folders have the same number of files
        self.image_filenames = sorted(os.listdir(image_folder))
        self.label_filenames = sorted(os.listdir(label_folder))

        assert len(self.image_filenames) == len(self.label_filenames), "Mismatch between image and label counts."

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # Load the input and label images
        image_path = os.path.join(self.image_folder, self.image_filenames[idx])
        label_path = os.path.join(self.label_folder, self.label_filenames[idx])

        image = Image.open(image_path) #.convert("RGB")
        label = Image.open(label_path) #.convert("RGB")

        # Ensure label size is 4x the input size
        input_size = image.size  # (width, height)
        label_size = label.size  # (width, height)
        expected_label_size = (input_size[0] * 4, input_size[1] * 4)

        assert label_size == expected_label_size, (
            f"Label size {label_size} does not match the expected size {expected_label_size} for input {input_size}."
        )

        # Apply transformations if provided
        if self.input_transform:
            image = self.input_transform(image)

        if self.label_transform:
            label = self.label_transform(label)

        return image, label

In [4]:
root_dir = "/kaggle/input/enhance-the-dark-world/archive"
train_dir = join(root_dir,"train")
val_dir = join(root_dir,"val")
test_dir = join(root_dir,"test")

In [5]:
train_dataset = ImageEnhancementDataset(
    join(train_dir,"train"), join(train_dir,"gt"),input_transform=image_transform,label_transform=label_transform
)

val_dataset = ImageEnhancementDataset(
    join(val_dir,"val"), join(val_dir, "gt"), input_transform=image_transform, label_transform=label_transform
)


In [6]:
batch_size = 4

train_loader = DataLoader(train_dataset,batch_size = batch_size, shuffle = True, num_workers = 4)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False, num_workers = 4)


In [7]:
for images,labels in train_loader:
    print(images.size())
    print(labels.size())
    break

torch.Size([4, 3, 160, 256])
torch.Size([4, 3, 640, 1024])


In [8]:
class Conv2d1x1(nn.Module):
    def __init__(self, input_channels: int, reduction_factor: int = 1, out_channels: int = None):
        super().__init__()

        if out_channels is None:
            out_channels = input_channels // reduction_factor

        self.out_channels = out_channels

        # define the 1x1 convolution layer
        self.conv = nn.Conv2d(in_channels=input_channels, out_channels=out_channels, kernel_size=(1, 1))

    def forward(self, x):
        return self.conv(x)


class DepthwiseConv2d(nn.Module):
    def __init__(self, input_channels: int, kernel_size: int):
        super().__init__()

        padding_size = kernel_size // 2

        self.conv = nn.Conv2d(in_channels=input_channels, out_channels=input_channels,
                              kernel_size=(kernel_size, kernel_size), groups=input_channels,
                              padding=(padding_size, padding_size))

    def forward(self, input_tensor):
        return self.conv(input_tensor)


class PointwiseConv2d(nn.Module):
    def __init__(self, input_channels: int, out_channels: int = None):
        super().__init__()

        if out_channels is None:
            out_channels = input_channels

        self.conv = Conv2d1x1(input_channels=input_channels, out_channels=out_channels)

    def forward(self, input_tensor):
        return self.conv(input_tensor)


class TwoFoldAttentionModule(nn.Module):
    class ChannelUnit(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            self.in_channels = input_channels

            # we define a global average pooling layer that extracts first-order statistics of features
            self.global_avg_pooling = nn.AdaptiveAvgPool2d(output_size=1)

            # input tensor
            conv_1x1_input_channels = input_channels // 2
            self.conv1x1_1 = Conv2d1x1(input_channels=conv_1x1_input_channels)
            self.conv1x1_2 = Conv2d1x1(input_channels=conv_1x1_input_channels)

        def forward(self, input_tensor):
            first_order_statistics = self.global_avg_pooling(input_tensor)  # output_size = (N, in_channels, 1, 1)
            half_channels = self.in_channels // 2
            first_half_input, second_half_input = torch.split(first_order_statistics,
                                                              split_size_or_sections=half_channels, dim=1)
            first_half_output = self.conv1x1_1(first_half_input)  # output_size = (N, in_channels/2, 1, 1)
            second_half_output = self.conv1x1_2(second_half_input)  # output_size = (N, in_channels/2, 1, 1)
            concatenated_halves = torch.cat((first_half_output, second_half_output), dim=1)
            output = torch.mul(concatenated_halves, input_tensor)  # output_size = (N, in_channels, 1, 1)
            return output

    class PositionalUnit(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()
            self.avg_pooling = nn.AvgPool2d(kernel_size=(7, 7))
            self.max_pooling = nn.MaxPool2d(kernel_size=(7, 7))
            kernel_size = 7
            padding_size = kernel_size // 2
            self.conv2d_1 = nn.Conv2d(in_channels=input_channels * 2, out_channels=input_channels,
                                      kernel_size=(kernel_size, kernel_size),
                                      padding=(padding_size, padding_size))

        def forward(self, input_tensor):
            height = input_tensor.size()[2]
            width = input_tensor.size()[3]
            output_max_pool = self.max_pooling(input_tensor)
            output_avg_pool = self.avg_pooling(input_tensor)
            output_pool = torch.cat((output_max_pool, output_avg_pool), dim=1)
            upsampled_out = F.interpolate(output_pool, size=(height, width), mode="bilinear", align_corners=False)
            output = self.conv2d_1(upsampled_out)
            return output

    def __init__(self, input_channels: int):
        super().__init__()
        self.conv1x1_1 = Conv2d1x1(input_channels=input_channels, reduction_factor=16)
        self.ca_unit = self.ChannelUnit(input_channels=self.conv1x1_1.out_channels)
        self.pos_unit = self.PositionalUnit(input_channels=self.conv1x1_1.out_channels)
        self.conv1x1_2 = Conv2d1x1(input_channels=self.conv1x1_1.out_channels, out_channels=input_channels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_tensor):
        conv_1x1_1_out = self.conv1x1_1(input_tensor)
        ca_unit_out = self.ca_unit(conv_1x1_1_out)
        pos_unit_out = self.pos_unit(conv_1x1_1_out)
        sum_output = torch.add(ca_unit_out, pos_unit_out)
        conv_1x1_2_out = self.conv1x1_2(sum_output)
        sigmoid_out = self.sigmoid(conv_1x1_2_out)
        output = torch.mul(input_tensor, sigmoid_out)
        return output


class AdaptiveResidualBlock(nn.Module):
    class BottleneckPath(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            self.dw_conv_1 = DepthwiseConv2d(input_channels=input_channels, kernel_size=3)

            self.pw_conv_1 = PointwiseConv2d(input_channels=input_channels)
            self.lrelu_1 = nn.LeakyReLU()

            self.dw_conv_2 = DepthwiseConv2d(input_channels=input_channels, kernel_size=3)

            self.tfam = TwoFoldAttentionModule(input_channels=input_channels)
            self.lrelu_2 = nn.LeakyReLU()
            self.pw_conv_2 = PointwiseConv2d(input_channels=input_channels)

        def forward(self, input_tensor):
            dw_conv_1_out = self.dw_conv_1(input_tensor)
            pw_conv_1_out = self.pw_conv_1(dw_conv_1_out)
            lrelu_1_out = self.lrelu_1(pw_conv_1_out)

            dw_conv_2_out = self.dw_conv_2(lrelu_1_out)
            tfam_out = self.tfam(dw_conv_2_out)
            lrelu_2_out = self.lrelu_2(tfam_out)
            output = self.pw_conv_2(lrelu_2_out)
            return output

    class AdaptivePath(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            self.global_avg_pooling = nn.AdaptiveAvgPool2d(output_size=1)

            self.pw_conv = PointwiseConv2d(input_channels=input_channels)

        def forward(self, input_tensor):
            global_avg_out = self.global_avg_pooling(input_tensor)
            output = self.pw_conv(global_avg_out)
            return output

    class ResidualPath(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()
            self.dw_conv = DepthwiseConv2d(input_channels=input_channels, kernel_size=3)

        def forward(self, input_tensor):
            return self.dw_conv(input_tensor)

    def __init__(self, input_channels: int):
        super().__init__()

        self.bn_path = self.BottleneckPath(input_channels=input_channels)

        self.ad_path = self.AdaptivePath(input_channels=input_channels)

        self.res_path = self.ResidualPath(input_channels=input_channels)

    def forward(self, input_tensor):
        bn_path_out = self.bn_path(input_tensor)

        sum_bn_input = torch.add(input_tensor, bn_path_out)

        res_path_out = self.res_path(sum_bn_input)

        ad_path_out = self.ad_path(input_tensor)

        output = torch.add(res_path_out, ad_path_out)
        return output


class ResidualConcatenationBlock(nn.Module):
    def __init__(self, input_channels: int):
        super().__init__()

        self.arb = AdaptiveResidualBlock(input_channels=input_channels)

        first_conv_input_channels = input_channels * 2
        self.conv_1x1_1 = Conv2d1x1(input_channels=first_conv_input_channels, out_channels=input_channels)

        second_conv_input_channels = input_channels * 3
        self.conv_1x1_2 = Conv2d1x1(input_channels=second_conv_input_channels, out_channels=input_channels)

        third_conv_input_channels = input_channels * 4
        self.conv_1x1_3 = Conv2d1x1(input_channels=third_conv_input_channels, out_channels=input_channels)

    def forward(self, input_tensor):
        arb_1_out = self.arb(input_tensor)

        concat_1_out = torch.cat((input_tensor, arb_1_out), dim=1)

        conv_1x1_1_out = self.conv_1x1_1(concat_1_out)

        arb_2_out = self.arb(conv_1x1_1_out)

        concat_2_out = torch.cat((concat_1_out, arb_2_out), dim=1)

        conv_1x1_2_out = self.conv_1x1_2(concat_2_out)

        arb_3_out = self.arb(conv_1x1_2_out)

        concat_3_out = torch.cat((concat_2_out, arb_3_out), dim=1)

        output = self.conv_1x1_3(concat_3_out)

        return output


class ResidualModule(nn.Module):
    def __init__(self, input_channels: int):
        super().__init__()

        self.rcb_1 = ResidualConcatenationBlock(input_channels=input_channels)

        first_conv_input_channels = input_channels * 2
        self.conv_1x1_1 = Conv2d1x1(input_channels=first_conv_input_channels, out_channels=input_channels)

        self.rcb_2 = ResidualConcatenationBlock(input_channels=input_channels)

        second_conv_input_channels = input_channels * 3
        self.conv_1x1_2 = Conv2d1x1(input_channels=second_conv_input_channels, out_channels=input_channels)

        self.rcb_3 = ResidualConcatenationBlock(input_channels=input_channels)

        third_conv_input_channels = input_channels * 4
        self.conv_1x1_3 = Conv2d1x1(input_channels=third_conv_input_channels, out_channels=input_channels)

    def forward(self, h_sfe):
        rcb_1_out = self.rcb_1(h_sfe)

        concat_1_out = torch.cat((h_sfe, rcb_1_out), dim=1)

        conv_1x1_1_out = self.conv_1x1_1(concat_1_out)

        rcb_2_out = self.rcb_2(conv_1x1_1_out)

        concat_2_out = torch.cat((concat_1_out, rcb_2_out), dim=1)

        conv_1x1_2_out = self.conv_1x1_2(concat_2_out)

        rcb_3_out = self.rcb_3(conv_1x1_2_out)

        concat_3_out = torch.cat((concat_2_out, rcb_3_out), dim=1)

        h_rm = self.conv_1x1_3(concat_3_out)

        return h_rm


class FeatureModule(nn.Module):
    def __init__(self, input_channels: int):
        super().__init__()

        # define the first layer, which is a TFAM
        self.tfam = TwoFoldAttentionModule(input_channels=input_channels)

        # define the second layer, which is a 3x3 conv layer
        kernel_size = 3
        padding_size = kernel_size // 2
        self.conv = nn.Conv2d(in_channels=input_channels, out_channels=input_channels,
                              kernel_size=(kernel_size, kernel_size), padding=padding_size)

    def forward(self, h_rm, h_sfe):
        # first, we feed the input tensor (h_rm) to the tfam layer
        tfam_out = self.tfam(h_rm)

        # then, we feed the output of the tfam layer to the convolutional layer
        h_gfe = self.conv(tfam_out)

        # finally, we compute the element-wise sum between the output of the convolutional layer and the shallow
        # features
        h_fm = torch.add(h_gfe, h_sfe)

        return h_fm


class UpNetModule(nn.Module):
    class Upsample2x(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            kernel_size = 3
            padding_size = kernel_size // 2

            # define the submodule that produces a feature map upsampled by 2x
            self.conv = nn.Conv2d(in_channels=input_channels, out_channels=input_channels * 4, kernel_size=(3, 3),
                                  padding=padding_size)
            self.pix_shuf = nn.PixelShuffle(upscale_factor=2)

        def forward(self, input_tensor):
            # feed the input tensor to the conv layer
            conv_out = self.conv(input_tensor)

            # feed the output of the conv layer to the pixel shuffle layer
            pix_shuf_out = self.pix_shuf(conv_out)

            return pix_shuf_out

    class Upsample3x(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            kernel_size = 3
            padding_size = kernel_size // 2

            # define the submodule that produces a feature map upsampled by 3x
            self.conv = nn.Conv2d(in_channels=input_channels, out_channels=input_channels * 9, kernel_size=(3, 3),
                                  padding=padding_size)
            self.pix_shuf = nn.PixelShuffle(upscale_factor=3)

        def forward(self, input_tensor):
            # feed the input tensor to the conv layer
            conv_out = self.conv(input_tensor)

            # feed the output of the conv layer to the pixel shuffle layer
            pix_shuf_out = self.pix_shuf(conv_out)

            return pix_shuf_out

    class Upsample4x(nn.Module):
        def __init__(self, input_channels: int):
            super().__init__()

            # define the first submodule that produces a feature map upsampled by 4x
            self.upsample_4x = nn.Sequential(UpNetModule.Upsample2x(input_channels=input_channels),
                                             UpNetModule.Upsample2x(input_channels=input_channels))

        def forward(self, input_tensor):
            # feed the input tensor to the upsampler
            return self.upsample_4x(input_tensor)

    def __init__(self, input_channels: int):
        super().__init__()

        # define the submodule that produces a feature map upsampled by 2x
        self.upsample_2x = self.Upsample2x(input_channels=input_channels)

        # define the submodule that produces a feature map upsampled by 3x
        self.upsample_3x = self.Upsample3x(input_channels=input_channels)

        # define the submodule that produces a feature map upsampled by 3x
        self.upsample_4x = self.Upsample4x(input_channels=input_channels)

    def forward(self, h_fm, scale: int):
        # feed the input tensor to one of the upsamplers according to the given scale
        if scale == 2:
            upsampled = self.upsample_2x(h_fm)
        elif scale == 3:
            upsampled = self.upsample_3x(h_fm)
        elif scale == 4:
            upsampled = self.upsample_4x(h_fm)
        else:
            raise Exception(f"Scale factor {scale} is invalid, select between 2, 3 or 4")

        return upsampled


class MultiPathResidualNetwork(nn.Module):
    def __init__(self, input_channels: int, n_features: int = 64):
        super().__init__()

        # initialize initial shallow feature extractor
        kernel_size = 3
        padding_size = kernel_size // 2
        self.sfe = nn.Conv2d(in_channels=input_channels, out_channels=n_features, kernel_size=(3, 3),
                             padding=padding_size)

        # define the Residual Module
        self.rm = ResidualModule(input_channels=n_features)

        # define the Feature Module
        self.fm = FeatureModule(input_channels=n_features)

        # define teh UpNet Module
        self.upnet = UpNetModule(input_channels=n_features)

        # define the final 3x3 convolution that restores the channels to three RGB channels
        self.out_conv = nn.Conv2d(in_channels=n_features, out_channels=input_channels, kernel_size=(3, 3),
                                  padding=padding_size)

    def forward(self, lrs, scale: int):
        # input is the batch of low resolution images, with shape (N, 3, 64, 64)
        h_sfe = self.sfe(lrs)  # output size (N, 64, 64, 64)

        # feed h_sfe to the residual module
        h_rm = self.rm(h_sfe)  # output size (N, 64, 64, 64)

        # feed h_rm and h_sfe to the feature module
        h_fm = self.fm(h_rm, h_sfe)  # output size (N, 64, 64, 64)

        # feed h_fm to the upnet module
        upscaled_fm = self.upnet(h_fm, scale)  # output size (N, 64, 64 * scale, 64 * scale)

        # feed upscaled feature map to the last 3x3 conv layer to get the final hr image in 3 RGB channels
        srs = self.out_conv(upscaled_fm)  # output size (N, 3,  64 * scale, 64 * scale)

        return srs

In [9]:
from torch.nn import DataParallel


# Create the model
model = MultiPathResidualNetwork(input_channels=3, n_features=128).to("cuda")

model = DataParallel(model)


# Test input
# lrs = torch.randn(1, 3, 160,160).to("cuda")  # Batch of 2 images, 3 channels, 64x64 resolution
lrs = torch.randn(1,3,160,256)
scale = 4

# Forward pass
with torch.no_grad():
    srs = model(lrs, scale)

print("Input shape:", lrs.shape)
print("Output shape:", srs.shape)
assert srs.shape == (1, 3, 160 * scale, 256 * scale), "Output shape is incorrect!"

Input shape: torch.Size([1, 3, 160, 256])
Output shape: torch.Size([1, 3, 640, 1024])


In [10]:
from torchmetrics import PeakSignalNoiseRatio

def train_one_epoch(model, train_loader, criterion, psnr_metric, optimizer, device):

    model.train()
    total_loss = 0.0
    total_psnr = 0.0
    
    progress_bar = tqdm(train_loader, desc='Training', unit='batch')
    for lr_images, hr_images in progress_bar:
        # Move data to the correct device
        lr_images = lr_images.to(device)
        hr_images = hr_images.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        sr_images = model(lrs=lr_images, scale=4)
        
        # Calculate loss
        loss = criterion(sr_images, hr_images)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Calculate PSNR for monitoring
        batch_psnr = psnr_metric(sr_images, hr_images).mean().item()
        
        # Update progress bar and tracking metrics
        total_loss += loss.item()
        total_psnr += batch_psnr
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}', 
            'PSNR': f'{batch_psnr:.2f}'
        })
    
    # Return average loss and PSNR for the epoch
    return total_loss / len(train_loader), total_psnr / len(train_loader)



def validate(model, val_loader, criterion, psnr_metric, device):

    model.eval()
    total_loss = 0.0
    total_psnr = 0.0
    
    progress_bar = tqdm(val_loader, desc='Validation', unit='batch')
    with torch.no_grad():
        for lr_images, hr_images in progress_bar:
            # Move data to the correct device
            lr_images = lr_images.to(device)
            hr_images = hr_images.to(device)
            
            # Forward pass
            sr_images = model(lrs=lr_images, scale=4)
            
            # Calculate loss
            loss = criterion(sr_images, hr_images)
            
            # Calculate PSNR for monitoring
            batch_psnr = psnr_metric(sr_images, hr_images).mean().item()
            
            # Update tracking metrics
            total_loss += loss.item()
            total_psnr += batch_psnr
            
            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}', 
                'PSNR': f'{batch_psnr:.2f}'
            })
    
    # Return average loss and PSNR for validation
    return total_loss / len(val_loader), total_psnr / len(val_loader)

def train_super_resolution_model(
    model, 
    train_loader, 
    val_loader, 
    num_epochs=3, 
    learning_rate=1e-4, 
    weight_decay=1e-5,
    checkpoint_path="best_model.pth"
):

    # Ensure model is on CUDA
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Loss function (typically L1 or L2 loss for image reconstruction)
    criterion = nn.MSELoss()
    
    # PSNR metric
    psnr_metric = PeakSignalNoiseRatio().to(device) # torchmetrics.image.PeakSignalNoiseRatio().to(device)
    
    # Optimizer with weight decay (L2 regularization)
    optimizer = optim.Adam(
        model.parameters(), 
        lr=learning_rate, 
        weight_decay=weight_decay
    )
    
    # Learning rate scheduler (optional, but often helpful)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='min', 
        factor=0.5, 
        patience=5
    )
    
    # Track best model
    best_val_psnr = float('-inf')
    best_epoch = 0
    
    # Training history
    history = {
        'train_loss': [],
        'train_psnr': [],
        'val_loss': [],
        'val_psnr': []
    }
    
    # Start training loop
    for epoch in range(num_epochs):
        print(f"\nEpoch [{epoch+1}/{num_epochs}]")
        
        # Train the model for one epoch
        train_loss, train_psnr = train_one_epoch(
            model, train_loader, criterion, psnr_metric, optimizer, device
        )
        history['train_loss'].append(train_loss)
        history['train_psnr'].append(train_psnr)
        
        # Validate the model
        val_loss, val_psnr = validate(model, val_loader, criterion, psnr_metric, device)
        history['val_loss'].append(val_loss)
        history['val_psnr'].append(val_psnr)
        
        # Print the epoch stats
        print(f"Training Loss: {train_loss:.4f}, Training PSNR: {train_psnr:.2f}")
        print(f"Validation Loss: {val_loss:.4f}, Validation PSNR: {val_psnr:.2f}")
        
        # Update learning rate scheduler based on validation loss
        scheduler.step(val_loss)

        torch.cuda.empty_cache()
        
        # Save the best model (based on validation PSNR)
        if val_psnr > best_val_psnr:
            best_val_psnr = val_psnr
            best_epoch = epoch
            print(f"New best model found, saving model at epoch {epoch+1}")
            torch.save(model.state_dict(), checkpoint_path)
    
    print(f"\nTraining completed. Best validation PSNR: {best_val_psnr:.2f} at epoch {best_epoch+1}")
    
    # Return the training history
    return history

In [11]:
history = train_super_resolution_model(
    model, train_loader, val_loader, num_epochs=100
)

/opt/conda/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `PeakSignalNoiseRatio` from `torchmetrics` was deprecated and will be removed in 2.0. Import `PeakSignalNoiseRatio` from `torchmetrics.image` instead.
  _future_warning(



Epoch [1/100]


Validation: 100%|██████████| 67/67 [00:18<00:00,  3.54batch/s, Loss=0.0002, PSNR=26.72]


Training Loss: 0.0013, Training PSNR: 25.98
Validation Loss: 0.0003, Validation PSNR: 25.84
New best model found, saving model at epoch 1

Epoch [2/100]


Validation: 100%|██████████| 67/67 [00:18<00:00,  3.54batch/s, Loss=0.0002, PSNR=27.74]


...

Training Loss: 0.0004, Training PSNR: 29.12
Validation Loss: 0.0002, Validation PSNR: 27.08

Epoch [99/100]


Validation: 100%|██████████| 67/67 [00:18<00:00,  3.54batch/s, Loss=0.0002, PSNR=28.08]


Training Loss: 0.0004, Training PSNR: 29.12
Validation Loss: 0.0002, Validation PSNR: 27.08

Epoch [100/100]


Validation: 100%|██████████| 67/67 [00:18<00:00,  3.53batch/s, Loss=0.0002, PSNR=28.08]


Training Loss: 0.0004, Training PSNR: 29.12
Validation Loss: 0.0002, Validation PSNR: 27.08

Training completed. Best validation PSNR: 27.12 at epoch 10


In [12]:
import os
from PIL import Image
import torch
from torchvision import transforms
import torchvision.utils as vutils
from tqdm import tqdm

def process_and_save_images(model, input_folder, output_folder, device):

    # Create output folder if it does not exist
    os.makedirs(output_folder, exist_ok=True)

    # Get list of image files in the input folder
    image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # Ensure the model is in evaluation mode
    model.eval()

    # Apply the image transformation to each image in the input folder with tqdm for progress bar
    for image_file in tqdm(image_files, desc="Processing images", unit="image"):
        image_path = os.path.join(input_folder, image_file)
        output_path = os.path.join(output_folder, image_file)

        # Open the image
        image = Image.open(image_path).convert('RGB')
        # image = Image.open(image_path).convert("L")

        # Apply the predefined image transformation
        transformed_image = image_transform(image).unsqueeze(0).to(device)

        # Send the image through the model to get the super-resolved image
        with torch.no_grad():
            sr_image = model(lrs=transformed_image, scale=4)  # Use your model's method

        # Convert the output image to CPU and denormalize if necessary (depending on how the model was trained)
        sr_image = sr_image.squeeze(0).cpu()
        
        # Assuming the output is a tensor, save it as an image
        vutils.save_image(sr_image, output_path)

        # Print message after saving
        # tqdm.write(f"Saved super-resolved image to {output_path}")

# Example usage
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Assume `model` is already defined and loaded with weights
process_and_save_images(model, "/kaggle/input/enhance-the-dark-world/archive/test", "/kaggle/working/output", device)

Processing images: 100%|██████████| 60/60 [00:25<00:00,  2.32image/s]


In [13]:
import os
import numpy as np
import pandas as pd
from PIL import Image

def images_to_csv(folder_path, output_csv):
    data_rows = []
    for filename in sorted(os.listdir(folder_path)):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path).convert('L') 
            image_array = np.array(image).flatten()[::8]
            data_rows.append(["gt"+filename.split('.')[0][4:], *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(output_csv, index=False)
    print(f'Successfully saved to {output_csv}')

folder_path = '/kaggle/working/output'
output_csv = 'submission.csv'
images_to_csv(folder_path, output_csv)

Successfully saved to submission.csv
